**性能评估**
本节将搭建性能评估环境，并对初始模型的性能指标进行验证，包括单卡吞吐量（tokens/s）和显存占用（GB）等。具体涵盖以下几个方面：
1. **软硬件环境**：对评估所使用的硬件配置和软件环境进行详细说明，确保实验的可重复性。
2. **程序运行时间精确测量**：介绍如何精确测量程序运行时间，以获取模型运行效率的准确数据。
3. **PyTorch性能分析器**：运用PyTorch性能分析器，深入分析模型在计算过程中的性能表现。
4. **GPU专业分析工具**：利用专业的GPU分析工具，对模型在GPU上的运行情况进行全方位的剖析。
5. **CPU性能分析工具**：借助CPU性能分析工具，评估模型在CPU上的性能指标，为全面了解模型性能提供多维度数据支持。 

# 1 软硬件环境

## 1.1 硬件资源
**GPU：**为了较少其他应用程序的干扰，尽量关闭其他显存占用。

```bash
nvidia-smi # 查看其它使用显卡的进程 查PID
ps aus |grep <PID> 查看进程详情
```

**CPU：** 查看CPU使用情况

```bash
htop # 查看 cpu使用情况, 核心利用情况，交换率等
```

## 1.2 设置各种随机种子

**python：**因为python生态没有不能统一设置随机种子，直接看代码

```python
torch.manual_seed(seed) # 这样的设置会使 Dropout 会在相同的位置丢弃神经元
np.random.seed(seed) # numpy

random.seed(seed) # python
# 👆这个不能完全设置随机性，随机来源很多，Hash不会因为这个设置固定

```

In [ ]:
import random

random.seed(42) # jupyter 里面每次执行结果一直，但是每次重启jupyter notebook内核，结果会变
print("Hash:", hash("hello")) # 
# 解决 Hash 的随机性需要设置环境变量
import os
os.environ['PYTHONHASHSEED'] = '0'

Hash: 6292224222569543665


**glob 模块：**glob 模块无法保证获取的文件顺序每次都一致,如果需要保证一致性，则需要手动排序（未验证）。

In [3]:
# glob 随机性测试
import glob
import random
# 固定种子
random.seed(42)

# 获取文件列表
files = glob.glob("/home/fl/code/python/FGPT/*.ipynb")

# 打印文件列表
print(files)

['/home/fl/code/python/FGPT/01 GPT.ipynb', '/home/fl/code/python/FGPT/02 Evaluate.ipynb']


**GPU 算子：** 约束 GPU 算子随机性，GPU 的算子的随机性来源很多，比如计算精度

比如结合律（A+B）\*C 和 (A \*C) + (B \*C) 不一定相等

又比如cuDnn 提供了多种卷积算法，默认会自动选择最优算法，这会导致卷积的结果不一致

一般通过以下方式约束随机性，但是添加设置会影响 GPU 性能,仅在调试分析时使用

```python
torch.backends.cudnn.deterministic = True # 约束算子底层实现的随机性
torch.backends.cudnn.benchmark = False # 约束 cuDnn算子随机选择算法
```


综上，一个完整的约束随机性的步骤集合如下：

```python
import torch
import random
import numpy as np
import glob
import os

def set_seed(seed):
    # 设置随机种子
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

    # 约束 GPU 算子随机性
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

## 1.3 限制频率

**GPU 频率限制：**GPU 会根据运行状态，自动调整显存频率和基础频率

GPU 锁频脚本如下(需要 GPU 的版本支持)

```bash
# 查询
nvidia-smi --query-gpu=pstate,clock,mem,clocks,sm,clocks.gr --format=csv

# 查询 GPU 支持的 Clock 频率组合
nvidia-smi --query-supported-clock=gpu_name,mem,gr --format=csv

# 设置 GPU 频率
sudo nvidia-smi -pm 1

# 固定 GPU 时钟频率
nvidia-smi -ac xxxx,xxxx #(mermory clock, graphics clock)
```

**CPU 频率：** CPU 的性能会被划分为不同的等级称为性能状态（P-state），可以通过工具包固定 p-state 和 CPU 频率
```bash
# 安装工具班、啊、包
sudo apt install cpufrequtils
# 设置最大/最小频率
sudo cpufrequtils -r -g performance # 设置 CPU 频率模式为 performance, -r 重新加载配置, -g 指定模式
sudo cpufrequtils -r -d 2.4GHz -u 2.4GHz # 设置 CPU 最大/最小频率

# 验证是否生效
cpufreq-info

# 或者直接查看
cat /sys/device/system/cpu/cpu0/cpufreq/scaling_cpu_freq
cat /sys/device/system/cpu/cpu0/cpufreq/scaling_max_freq
cat /sys/device/system/cpu/cpu0/cpufreq/scaling_min_freq
```
**内存频率：** 内存频率一般不会影响程序性能，但是会影响到显存带宽，显存带宽会影响 GPU 性能，所以需要固定内存频率

```bash
# 查询

```


# 2 时间测量

# 3 Pytorch性能分析

# 4 GPU分析工具

# 5 CPU 性能分析工具